In [ ]:
import numpy as np
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
import pickle
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
to_categorical = tf.keras.utils.to_categorical

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Mask Detection

/content/drive/My Drive/Mask Detection


In [ ]:
train_path =  os.path.join('dest_folder/train')
test_path =  os.path.join('dest_folder/test')
val_path =  os.path.join('dest_folder/val')

In [ ]:
training_data = []
category = ["with_mask", "without_mask"]

for i in os.listdir(train_path):
    label = category.index(i)
    new_path = os.path.join(train_path, i)
    for j in os.listdir(new_path):
      try:
          img_to_array = cv2.imread(os.path.join(new_path, j))
          new_array = cv2.resize(img_to_array, (150,200))
          training_data.append([new_array, label])
      except:
          pass

In [ ]:
print(len(training_data))

1315


In [ ]:
pickle_out = open('training_data.pickle', 'wb')
pickle.dump(training_data, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open('training_data.pickle', 'rb')
training_data_pickle = pickle.load(pickle_in)

In [ ]:
X = []
y = []

for features, labels in training_data_pickle:
    X.append(features)
    y.append(labels)

In [ ]:
X = np.array(X)
y = np.array(y)

print(len(X))
print(len(y))
print(X.shape)
print(y.shape)

1315
1315
(1315, 200, 150, 3)
(1315,)


In [ ]:
# to categorical 
# y = to_categorical(y)

In [ ]:
# ingat reshape dulu baru normalisasi
X = np.array(X).reshape(-1,200,150,3)
X = X/255.0

In [ ]:
print(X.shape)

(1315, 200, 150, 3)


In [ ]:
import tensorflow as tf

resnet = tf.keras.applications.ResNet50V2(weights='imagenet', include_top=False, input_shape=(200,150,3))

In [ ]:
model = tf.keras.models.Sequential([
    resnet, 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

resnet.trainable = False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 7, 5, 2048)        23564800  
_________________________________________________________________
flatten (Flatten)            (None, 71680)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               36700672  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 60,265,985
Trainable params: 36,701,185
Non-trainable params: 23,564,800
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# from tensorflow.keras.callbacks import TensorBoard

# NAME = "Face_mask_recognition"
# tboard_log_dir = os.path.join("tensorboard",NAME)
# tensorboard = TensorBoard(log_dir = tboard_log_dir)

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get("accuracy") > 0.99):
            print(" Reached 99% Accuracy")
            self.model.stop_training = True
            
callbacks = myCallback()

In [ ]:
history = model.fit(X, y, batch_size=128, validation_split=0.2, epochs=30, callbacks=[callbacks])

Epoch 1/30
9/9 [==============================] - 97s 11s/step - loss: 1.0877 - accuracy: 0.9087 - val_loss: 3.6183 - val_accuracy: 0.9506
Epoch 2/30
9/9 [==============================] - 98s 11s/step - loss: 0.4382 - accuracy: 0.9905 - val_loss: 3.4543 - val_accuracy: 0.9544


In [ ]:
from tensorflow.keras.preprocessing import image

# dibawah ini nama nya hanya perumpamaan saja
category = ["with_mask", "without_mask"]

for i in os.listdir(val_path):
  path = os.path.join(val_path, i)
  for j in os.listdir(path):
    img = image.load_img(os.path.join(path, j), target_size=(200,150), color_mode="rgb")
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x/255.0

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    result = []

    if classes[0] > 0.5:
      temp = f"{classes[0]} From Validation Class " + i + " -> Without Mask"

    else:
      temp = f"{classes[0]} From Validation Class " + i + " -> With Mask"
    
    result.append(temp)
    print(result)

['[1.] From Validation Class without_mask -> Without Mask']
['[1.] From Validation Class without_mask -> Without Mask']
['[1.] From Validation Class without_mask -> Without Mask']
['[1.] From Validation Class without_mask -> Without Mask']
['[1.] From Validation Class without_mask -> Without Mask']
['[1.] From Validation Class without_mask -> Without Mask']
['[1.] From Validation Class without_mask -> Without Mask']
['[1.] From Validation Class without_mask -> Without Mask']
['[1.] From Validation Class without_mask -> Without Mask']
['[1.] From Validation Class without_mask -> Without Mask']
['[1.] From Validation Class without_mask -> Without Mask']
['[1.] From Validation Class without_mask -> Without Mask']
['[1.] From Validation Class without_mask -> Without Mask']
['[1.] From Validation Class without_mask -> Without Mask']
['[1.] From Validation Class without_mask -> Without Mask']
['[1.] From Validation Class without_mask -> Without Mask']
['[1.] From Validation Class without_mas

In [ ]:
model.save("face_mask_trained.h5")